<a href="https://colab.research.google.com/github/Vanitha-Jain/capstone/blob/Molecular-descriptors/RDKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit-pypi


In [ ]:
!pip install mordred

In [ ]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [ ]:
dataset=pd.read_csv('/content/filtered_bioactivity_data.csv')

In [ ]:
df=dataset.dropna(subset=['canonical_smiles'])

In [ ]:
df

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL167155,CC(=O)c1ccc(-c2cc(NS(=O)(=O)c3ccccc3C)c(C(=O)O...,inactive,415.492,4.42512,2,5,4.920819
1,CHEMBL166740,Cc1ccccc1S(=O)(=O)Nc1cc(-c2ccc(S(C)(=O)=O)cc2)...,inactive,451.547,3.62602,2,6,4.000000
2,CHEMBL165038,Cc1cc(S(=O)(=O)Nc2cc(-c3ccccc3)sc2C(=O)O)c(C)c...,inactive,421.927,5.18434,2,4,4.853872
3,CHEMBL435857,Cc1cc(Cl)ccc1S(=O)(=O)Nc1cc(-c2ccccc2)sc1C(=O)O,inactive,407.900,4.87592,2,4,4.886057
4,CHEMBL352783,Cc1cccc(S(=O)(=O)Nc2cc(-c3ccccc3)sc2C(=O)O)c1,inactive,373.455,4.22252,2,4,4.000000
...,...,...,...,...,...,...,...,...
1224,CHEMBL5429863,Nc1ccccc1/C=C/c1nccc2ccccc12,inactive,255.320,4.38650,0,1,4.397940
1225,CHEMBL5436920,C#Cc1ccc(/C=C/c2nccc3ccccc23)cc1,inactive,255.320,4.38650,0,1,4.890084
1226,CHEMBL5405698,C#Cc1ccccc1/C=C/c1nccc2ccccc12,inactive,307.396,6.07220,0,1,4.604848
1227,CHEMBL5416562,C(=C/c1nccc2ccccc12)\c1ccc(-c2ccccc2)cc1,inactive,228.247,2.97380,3,3,4.397940


In [ ]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors = []
    for mol in mols:
        # add hydrogens to molecules
        mol = Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors, desc_names

# Split the SMILES into chunks of 100,000 for faster processing
chunk_size = 100000
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

total_chunks=len(chunks)
total_time=0

# Check if there is an existing output file
if os.path.isfile('RDkit_descriptors.csv'):
    existing_data = pd.read_csv('RDkit_descriptors.csv', index_col=0)
else:
    existing_data = pd.DataFrame()

# Calculate descriptors for each chunk and concatenate the results
for i, chunk in enumerate(tqdm(chunks, desc='Processing', total=len(chunks))):
    # Check if this chunk has already been processed
    if len(existing_data) >= len(chunk):
        continue
    # Calculate descriptors for this chunk
    descriptors, desc_names = RDkit_descriptors(chunk['canonical_smiles'])
    # Convert the descriptors to a dataframe
    df_with_200_descriptors = pd.DataFrame(descriptors, columns=desc_names,)
    # Add the chunk index as a new column
    df_with_200_descriptors['chunk_index'] = i
    # Append the data to the existing data
    existing_data = pd.concat([existing_data, df_with_200_descriptors], axis=0)
    # Save the data after each chunk
    existing_data.to_csv('RDkit_descriptors.csv')

# Save the final data
existing_data.to_csv('RDkit_descriptors.csv')

Processing: 100%|██████████| 1/1 [01:38<00:00, 98.17s/it]
